<a href="https://colab.research.google.com/github/NavitB/IRProject/blob/main/run_frontend_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [4]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2022-01-06 21:17:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  8.34MB/s    in 1.6s    

2022-01-06 21:17:06 (8.34 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip


In [5]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
# !./ngrok authtoken YOUR_NGROK_TOKEN_HERE

In [6]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

In [7]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math
import builtins
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
--2022-01-06 21:17:26--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.32.87.12, 13.32.87.63, 13.32.87.68, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.32.87.12|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [9]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [10]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [11]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [12]:
pandasDf = pd.read_csv("/content/newdata.csv")
pandasDf = pandasDf.drop(columns = "Unnamed: 0")
df_data = spark.createDataFrame(pandasDf)
# df_data_rdd = df_data.rdd
# print(df_data_rdd.take(1))
id_text = df_data.select('id','text')
id_title = df_data.select('id','title')
id_anchor = df_data.select('id', 'anchor_text')
# id_text = id_text.toPandas()
# lst = [id_text.loc[id_text['id'] == 723126]['text']]
# print(lst)


id_text = id_text.rdd
id_title = id_title.rdd
id_anchor = id_anchor.rdd
# print(id_anchor.take(2))
# print(id_text.take(1))
# print(id_title.take(5))
inverted_text = InvertedIndex()
inverted_title = InvertedIndex()

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(id, text):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  tuples = []
  tf_dict = Counter(tokens)
  res = []
  [res.append(x) for x in tokens if x not in res]
  for t in res:
    tuples.append((t,(id,tf_dict[t])))
  return tuples

In [14]:
def count_doc_len(doc_id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (doc_id,len(tokens))



In [15]:
def tf_for_term_id(doc_id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  tuples = []
  tf_dict = Counter(tokens)
  res = []
  [res.append(x) for x in tokens if x not in res]
  for t in res:
    tuples.append((t,tf_dict[t]))
  return tuples

In [16]:
word_counts_text = id_text.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_title = id_title.flatMap(lambda x: word_count(x[0], x[1]))

len_docs_text = id_text.map(lambda x: count_doc_len(x[0], x[1]))
len_docs_text = len_docs_text.collectAsMap()
inverted_text.DL = len_docs_text

len_docs_title = id_title.map(lambda x: count_doc_len(x[0], x[1]))
len_docs_title = len_docs_title.collectAsMap()
inverted_title.DL = len_docs_title



anchor_text = id_anchor.map(lambda r : r['anchor_text'])
# # .reduce(lambda x, y : x['text'] + y['text'])
# # print(
# one = anchor_text.take(1)
# print(one)
# # flat_anchor = id_anchor.flatMap(lambda x : x.flatMap)
# numbers = sc.parallelize([1,7,8,9,5,77,48])
# print(numbers.take(5))
# print(id_anchor.take(2))
# word_counts_anchor = id_anchor.flatMap(lambda x: word_count(x[0], x[1].reduce(lambda x,y : x +" " + y)))
# # print(word_counts_title.take(1))
# # print(word_counts_anchor.take(1))
# #print(word_counts_anchor.take(1))

In [17]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)

In [18]:
#create postings and sort by doc_id

print(word_counts_text.take(5))
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

#anchor?
#filter out rare words - appearing in less than 10 docs
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>10)
postings_filtered_title = postings_title.filter(lambda x: len(x[1])>10)
#anchor?

[('time', (4055486, 8)), ('walker', (4055486, 6)), ('1982', (4055486, 3)), ('science', (4055486, 8)), ('fiction', (4055486, 4))]


In [19]:
from operator import add

total_terms_text = postings_filtered_text.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_text.term_total = total_terms_text.collectAsMap()

total_terms_title = postings_filtered_title.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_title.term_total = total_terms_title.collectAsMap()




In [20]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  rdd2 = postings.map(lambda x: (x[0], len(x[1]))) 
  return rdd2

In [21]:
w2df_text = calculate_df(postings_filtered_text)
w2df_title = calculate_df(postings_filtered_title)

w2df_text_dict = w2df_text.collectAsMap()
w2df_title_dict = w2df_title.collectAsMap()


In [22]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,base_dir):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  rdd2 = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey()
  rdd3 = rdd2.map(lambda x: InvertedIndex.write_a_posting_list(x,base_dir))
  return rdd3


In [23]:
#rdd for postings locations
!mkdir text_pl title_pl 
postings_locs_text = partition_postings_and_write(postings_filtered_text,'text_pl').collect()
postings_locs_title = partition_postings_and_write(postings_filtered_title,'title_pl').collect()


mkdir: cannot create directory ‘text_pl’: File exists
mkdir: cannot create directory ‘title_pl’: File exists


In [24]:
#create super set for allpotings locs
super_posting_locs_text = defaultdict(list)
for posting_loc in postings_locs_text:
  for k, v in posting_loc.items():
    super_posting_locs_text[k].extend(v)

super_posting_locs_title = defaultdict(list)
for posting_loc in postings_locs_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)


#anchor?

In [25]:
inverted_text.posting_locs = super_posting_locs_text
inverted_title.posting_locs = super_posting_locs_title

inverted_text.df = w2df_text_dict
inverted_title.df = w2df_title_dict

inverted_text.write_index('.', 'index_text')
inverted_title.write_index('.', 'index_title')
locs = inverted_title.posting_locs
DL = inverted_text.DL
N = len(DL)

In [33]:
print(DL[4055486])
lst = [('time', 8), ('walker', 6), ('1982', 3), ('science', 8), ('fiction', 4), ('horror', 4), ('film', 11), ('directed', 1)]

694


In [35]:
# def calculate_tfidf_doc(terms_tf, dl,N):
#   sum = 0
#   for term, tf in terms_tf:
#     df_term = 10
#     # w2df_text_dict[term]
#     idf = math.log10(N/int(df_term))
#     tf = tf/dl
#     tfidf = tf*idf
#     sum += math.pow(tfidf,2)
#   return sum

# print(calculate_tfidf_doc(lst, 20,200 ))
def calculate_tfidf_doc(tf_term_doc, index):
  # df_dict = index.df
  # N = len(index.DL)
  # DL_dict = index.DL

  # terms_tf = tf_term_doc.map(lambda x : list(x[1]))

  # tf_term_doc = tf_term_doc.flatMapValues(lambda x : x)
  # doc_term_tf = tf_term_doc.map(lambda x : {x[0] : x[1]})
  doc_term_tfidf = tf_term_doc.map(lambda x : (x[0] , builtins.sum([math.pow((tf/DL[x[0]]) * (math.log(N/w2df_text_dict[term],10)),2) for term, tf in x[1] if term in w2df_text_dict])))


  # doc_term_tfidf = tf_term_doc.map(lambda x : {x[1][1] : [(tf/DL_dict[doc_id] * math.log(N/len(x[1]),10)) for doc_id, tf in x[1]]})

#   # tfidf_docs_terms = tf_term_doc.map(lambda x : {x[0]: ((x[1][1]/DL_dict[x[0]]) * (math.log(N/df_dict[x[1][0]],10)))})
#   # tfidf_docs = tfidf_docs_terms.map(lambda x : (x[0], x[1][1])).map(lambda x : (x[0], math.pow(x[1],2)))
#   # docID_rdd = tfidf_docs_terms.flatMap(lambda x: x[1].items()).groupByKey().mapValues(list(x[1]))
  return doc_term_tfidf

# # # ================================= Computing IDF====================================
# docs_num = len(inverted_text.DL)

# 	# ('term', (idf,[('document id1', tf1), ('document id2', tf2)....] ))
# idf = postings_filtered_text.map(lambda x: (x[0], (math.log10(docs_num/len(x[1])), x[1])))

# 	# # ================================ Computing TF *IDF====================================
	# # ('term', {'document id1': tf*idf1, 'document id2': tf*idf2 .....})   value is in dictionary form
# tfidf= idf.map(lambda x: (x[0], {w[0]: x[1][0] * w[1] for w in x[1][1]}))
# # docID_rdd = tfidf.flatMap(lambda x: x[1].items()).groupByKey().mapValues(reduce_word_counts)
# print(docID_rdd.take(1))

In [36]:
tf_term_doc_text = id_text.map(lambda x : (x[0], tf_for_term_id(x[0], x[1])))
print(tf_term_doc_text.take(1))
# diction = tf_term_doc_text.collectAsMap()
# print(diction)
DL = inverted_text.DL
N = len(DL)

# test = tf_term_doc_text.map(lambda x : (x[0], calculate_tfidf_doc(x[1], 200 , 5)))
# tfidf = tf_term_doc_text.flatMap(lambda x : (x[0], calculate_tfidf_doc(x[1], DL[x[0]] , N)))
# tfidf = tf_term_doc_text.groupByKey().mapValues(calculate_tfidf_doc(DL))
# print(test.take(10))


tfidf = calculate_tfidf_doc(tf_term_doc_text, inverted_text)
print(tfidf.collectAsMap())
# df_dict = w2df_text_dict
# print(dict(df_dict.items()))
# DL = inverted_text.DL
# print(DL)

# tf_idf = tf_term_doc_text.map(lambda x : (x[0], calculate_tfidf_doc(x[1],df_dict, DL[x[0]], len(DL))))
# print(tf_idf.take(1))

[(4055486, [('time', 8), ('walker', 6), ('1982', 3), ('science', 8), ('fiction', 4), ('horror', 4), ('film', 11), ('directed', 1), ('tom', 3), ('kennedy', 1), ('since', 1), ('release', 2), ('received', 1), ('negative', 2), ('reviews', 1), ('critics', 1), ('title', 2), ('another', 5), ('planet', 6), ('featured', 2), ('mystery', 5), ('theater', 5), ('3000', 5), ('episode', 7), ('405', 1), ('first', 2), ('aired', 2), ('july', 2), ('1992', 2), ('shown', 1), ('metv', 1), ('show', 1), ('svengoolie', 1), ('june', 1), ('2021', 1), ('plot', 1), ('california', 4), ('university', 4), ('sciences', 1), ('professor', 1), ('douglas', 11), ('mccadden', 1), ('explores', 1), ('tomb', 3), ('ancient', 2), ('egyptian', 3), ('king', 2), ('tutankhamun', 3), ('earthquake', 1), ('causes', 2), ('wall', 1), ('collapse', 1), ('revealing', 2), ('hidden', 1), ('chamber', 1), ('inside', 1), ('finds', 1), ('mummy', 9), ('sarcophagus', 4), ('unbeknownst', 1), ('body', 3), ('dead', 2), ('extraterrestrial', 1), ('alien'

In [61]:
# # print(tf_term_doc_text.take(2))
# N = len(inverted_text.DL)
# df_dict = inverted_text.df
# # print(df_dict)
# # tfidf_docs_text = tf_term_doc_text.map(lambda x : (x[0], calculate_tfidf_doc(x[1], df_dict, N)))
# # print(tfidf_docs_text.take(1))
# # print(tf_term_doc_text.flatMapValues(lambda x : x).take(4))
# # .groupByKey().mapValues(list).take(2))

# tfidf_docs_text = calculate_tfidf_doc(tf_term_doc_text, inverted_text)
# # print(tfidf_docs_text.groupByKey().mapValues(list).take(1))
# tfidf_docs_text_grouped = tfidf_docs_text.groupByKey().mapValues(list).collect()
# print(tfidf_docs_text.take(7))
# print(tfidf_docs_text_grouped)
# # print(tf_term_doc_text.flatMapValues(lambda x: x).take(2))


NameError: ignored

In [25]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [26]:
# pl_title = read_posting_list(inverted_title, 'mask')
# pl_text = read_posting_list(inverted_text, 'mask')
# df_title = len(pl_title)
# df_text = len(pl_text)

# # lst = [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462]
# lst = set([63631542, 821829, 633458, 63567907, 14208718, 7912947, 159546, 67759831, 7439323, 64192678, 40909056, 5322079, 63164437, 31728348, 12772, 1402997, 1156703, 1840653, 568164, 1814699, 2945076, 2565663, 442333, 33567480, 468313, 19718702, 987724, 903187, 1485962, 5003908, 4201044, 28086000, 28844729, 1265651, 4659608, 64966775, 55960921, 8970797, 4301719, 35892659, 2248622, 6939163, 48561519, 261396, 34298473, 12263290, 44258772, 25219375, 149426, 6558203, 46567337, 46784964, 6458321, 46576311, 18823362, 48315099, 56440599, 57159776, 74910, 67891964, 1702593, 560306, 1015304, 705756, 7479199, 57772096, 58946599, 1210300, 15716827, 34336876, 9040490, 439102, 264104, 11061915, 39774839, 34735506, 42174581, 66859257, 3293969, 16315, 23166476, 42812440, 66935753, 51990351, 36619752, 59863622, 38404, 64027932])
# print("len")
# print(len(lst))
# pl_id_title = [x[0] for x in pl_title]
# print(pl_id_title)
# pl_id_text = [x[0] for x in pl_text]
# print(pl_id_text)

# eq_title_text = len([x for x in pl_id_text if x in pl_id_title])
# print(eq_title_text)

# eq_text_lst = len([x for x in pl_id_text if x in lst])
# print(eq_text_lst)

# eq_title_lst = len([x for x in pl_id_title if x in lst])




In [27]:
def get_posting_gen(index):
    """
    This function returning the generator working with posting list.
    
    Parameters:
    ----------
    index: inverted index    
    """
    words, pls = zip(*index.posting_lists_iter())
    return words,pls

In [28]:
words, pls = get_posting_gen(inverted_text)

In [29]:
def generate_query_tfidf_vector(query_to_search,index):
    """ 
    Generate a vector representing the query. Each entry within this vector represents a tfidf score.
    The terms representing the query will be the unique terms in the index.

    We will use tfidf on the query as well. 
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the query.    

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.
    
    Returns:
    -----------
    vectorized query with tfidf scores
    """
    
    epsilon = .0000001
    total_vocab_size = len(index.term_total)
    Q = np.zeros((total_vocab_size))
    term_vector = list(index.term_total.keys()) 
    counter = Counter(query_to_search)
    for token in np.unique(query_to_search):
        if token in index.term_total.keys(): #avoid terms that do not appear in the index.               
            tf = counter[token]/len(query_to_search) # term frequency divded by the length of the query
            df = index.df[token]            
            idf = math.log((len(index.DL))/(df+epsilon),10) #smoothing
            try:
                ind = term_vector.index(token)
                Q[ind] = tf*idf                    
            except:
                pass
    return Q
    

In [30]:
 def get_candidate_documents_and_scores(query_to_search,index,words,pls):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score. 
    """
    candidates = {}
    N = len(index.DL)
    for term in np.unique(query_to_search):        
        if term in words:            
            list_of_doc = pls[words.index(term)]
            normlized_tfidf = [(doc_id,(freq/index.DL[doc_id])*math.log(N/index.df[term],10)) for doc_id, freq in list_of_doc]           
            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf               
    return candidates



In [31]:
def generate_document_tfidf_matrix(query_to_search,index,words,pls):
    """
    Generate a DataFrame `D` of tfidf scores for a given query. 
    Rows will be the documents candidates for a given query
    Columns will be the unique terms in the index.
    The value for a given document and term will be its tfidf score.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    DataFrame of tfidf scores.
    """
    
    total_vocab_size = len(index.term_total)
    candidates_scores = get_candidate_documents_and_scores(query_to_search,index,words,pls) #We do not need to utilize all document. Only the docuemnts which have corrspoinding terms with the query.
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)
    
    D.index = unique_candidates
    D.columns = index.term_total.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key    
        D.loc[doc_id][term] = tfidf


    return D
     


In [32]:
def cosine_similarity(D,Q):
    """
    Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
    Generate a dictionary of cosine similarity scores 
    key: doc_id
    value: cosine similarity score
    
    Parameters:
    -----------
    D: DataFrame of tfidf scores.

    Q: vectorized query with tfidf scores
    
    Returns:
    -----------
    dictionary of cosine similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: cosine similarty score.
    """
    dict_cos = {}
    for doc_id, row in D.iterrows(): 
      vec_doc = row.to_numpy()
      result = np.dot(vec_doc, Q)/(np.linalg.norm(vec_doc) * np.linalg.norm(Q))
      dict_cos[doc_id] = result
    return dict_cos


    

In [33]:
def get_top_n(sim_dict,N=3):
    """ 
    Sort and return the highest N documents according to the cosine similarity score.
    Generate a dictionary of cosine similarity scores 
   
    Parameters:
    -----------
    sim_dict: a dictionary of similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: similarity score. We keep up to 5 digits after the decimal point. (e.g., round(score,5))

    N: Integer (how many documents to retrieve). By default N = 3
    
    Returns:
    -----------
    a ranked list of pairs (doc_id, score) in the length of N.
    """
    lst = [(doc_id , builtins.round(score,5)) for doc_id, score in sim_dict.items()]
    return sorted(lst, key = lambda x: x[1], reverse=True)[:N]
    





In [34]:
def get_topN_score_for_queries(queries_to_search,index,N=3):
    """ 
    Generate a dictionary that gathers for every query its topN score.
    
    Parameters:
    -----------
    queries_to_search: a dictionary of queries as follows: 
                                                        key: query_id
                                                        value: list of tokens.
    index:           inverted index loaded from the corresponding files.    
    N: Integer. How many documents to retrieve. This argument is passed to the topN function. By default N = 3, for the topN function. 
    
    Returns:
    -----------
    return: a dictionary of queries and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id, score). 
    """
    top_N = {}
    for q_id, q in queries_to_search.items() :
      Q = generate_query_tfidf_vector(q,index)
      D = generate_document_tfidf_matrix(q, index, words, pls)
      top_N[q_id] = get_top_n(cosine_similarity(D, Q), N)
    return top_N


In [40]:
dict_q = {}
dict_q[1] = ['best', 'marvel', 'movie']
tfidf_queries_score_train = get_topN_score_for_queries(dict_q, inverted_text,100)
print(tfidf_queries_score_train)

# tfidf_titles = get_topN_score_for_queries(dict_q, inverted_title,100)
# print(tfidf_titles)


lst = [57069491, 65967176, 42163310, 878659, 27306717, 41677925, 1074657, 44240443, 17296107, 60952488, 43603241, 22114132, 46208997, 36450985, 41974555, 56289672, 60616450, 59502488, 33038861, 61699239, 61651800, 39368416, 29129051, 55935213, 54537218, 62372638, 60774345, 63090183, 37497391, 51430647, 67229718, 9110929, 61329320, 44254295, 41974496, 200563, 58481694, 48530084, 56289572, 22144990, 612052, 59162931, 55511148, 55511147, 61073786, 59892, 36484005, 36484254, 66423851, 62482816, 1275470, 5676692, 7927053, 60754840, 26999426, 60744481, 56289553, 60463979, 701741, 60283633, 1129847, 36439749, 4451883, 55511155, 22144721, 45359871, 723126, 43655965, 57275457, 12673434, 43867095, 26763420, 39293265, 15003874, 41668588, 61592102, 67063919, 11891433, 64057670, 61513780, 39345917, 67063906, 1221476, 41008758, 60587000, 7729, 2152196, 5027882, 509738, 403585, 26866372, 1339248, 3473503, 4148655]

lst_docs_returned = [x[0] for x in tfidf_queries_score_train[1]]

lst_equal = [x for x in lst_docs_returned if x in lst]
print(len(lst))
print(len(lst_equal))

lst_top_6 = [57069491, 65967176, 42163310, 878659, 27306717, 41677925]

lst_indecies = [lst_docs_returned.index(x) for x in lst_top_6]
print(lst_indecies)


{1: [(723126, 1.0), (2720954, 1.0), (22144721, 0.99775), (9110929, 0.9973), (36439749, 0.99648), (26866372, 0.99504), (22144990, 0.99249), (44240443, 0.99004), (56289572, 0.97823), (41974496, 0.97664), (56289672, 0.9754), (39345917, 0.97175), (56289553, 0.97157), (5676692, 0.96672), (175537, 0.96291), (29129051, 0.96138), (41974555, 0.96081), (55511148, 0.96062), (6458321, 0.95187), (32158923, 0.95187), (57275457, 0.95187), (65766671, 0.95187), (6837398, 0.95167), (1275470, 0.94983), (36450985, 0.94926), (59162931, 0.94824), (45359871, 0.94539), (59892, 0.9445), (36484254, 0.94372), (26999426, 0.94349), (1129847, 0.93895), (48530084, 0.93777), (65967176, 0.92834), (44254295, 0.92824), (54537218, 0.92383), (41668588, 0.9221), (51430647, 0.91723), (43867095, 0.9109), (55511147, 0.90739), (36484005, 0.90484), (55935213, 0.90427), (58150649, 0.90306), (60774345, 0.90033), (12673434, 0.9), (19347339, 0.8978), (169672, 0.89756), (60616450, 0.8962), (41677925, 0.89198), (559356, 0.89), (37497

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))